# Airport Security Simulation

In [1]:
# Build a simulation model of airport that mimics:
# - Passport control (triangular 30-45-90 seconds)
# - Security scan (constant 10 seconds)
# Passengers arrive randomly (iat normal 60 seconds)
# Luggage on belt at security scan, one at a time (20-40 seconds)
# Conveyor 10m, speed 0.5 m/s
# At security scan manual check for 10% of passengers with one person, others don't wait 
# After luggage, passengers take luggage again (20-40 seconds)
# Passenger movements 2.5km/h, 10m along luggage belt
# Distance between other processes 5m ????

# Model for 4 hours, warm-up 1 hour.
# Run 10 replications (average + 90% confidence interval)
# Provide utilization data of passport control, scanner and secondary check

In [2]:
import salabim as sim

## Working 15/09/2017

In [3]:
### Simulation without matching luggage to passenger

class PassengerGenerator(sim.Component):
    def process(self):
        while True:
            Passenger()
            yield self.hold(sim.Normal(60).sample())
            
class Passenger(sim.Component):
    def process(self):
        # Passport control
        self.enter(waitingline_passport)
        if passportcontrol.ispassive():
            passportcontrol.activate()
        yield self.passivate()
        
        # Walking from passport control to security scan
        yield self.hold(5/(2.5*1000/(60*60))) # convert 2.5 km/h to m/s
        
        # Security Scan
        self.enter(waitingline_security)
        if securityscan.ispassive():
            securityscan.activate()
        yield self.passivate()
        
        # Put luggage on belt
        yield self.hold(sim.Uniform(20,40).sample())
        Luggage(self.name())
        
        # 10% Pat Down Requirement
        if (sim.Uniform(1,100).sample() <= 10):
            # Walking from security scan to patdown
            yield self.hold(5/(2.5*1000/(60*60))) # convert 2.5 km/h to m/s
            # Patdown
            self.enter(waitingline_patdown)
            if patdown.ispassive():
                patdown.activate()
            yield self.passivate()
        
        # Walking along luggage belt
        yield self.hold(10/(0.5*1000/(60*60))) # convert 2.5 km/h to m/s
        
        # Pick up luggage, not matching owner
        yield self.hold(sim.Uniform(20,40).sample())
        waitingline_luggage.pop()
                
class Luggage(sim.Component): 
    def __init__(self, passengerName, *args, **kwargs): 
        super().__init__(*args, **kwargs)
        self.owner = passengerName
        
    def process(self):
        # Roll along luggage belt
        yield self.hold(10/(2.5*1000/(60*60))) # convert 2.5 km/h to m/s
        # Enter luggage waiting line
        self.enter(waitingline_luggage)
        
class PassportControl(sim.Component):
    def process(self):
        while True:
            while len(waitingline_passport) == 0:
                yield self.passivate()
            self.passenger = waitingline_passport.pop()
            yield self.hold(sim.Triangular(30.,90.,45.).sample())
            self.passenger.activate()
            
class SecurityScan(sim.Component):
    def process(self):
        while True:
            while len(waitingline_security) == 0:
                yield self.passivate()
            self.passenger = waitingline_security.pop()
            yield self.hold(10)
            self.passenger.activate()
            
class PatDown(sim.Component):
    def process(self):
        while True:
            while len(waitingline_patdown) == 0:
                yield self.passivate()
            self.passenger = waitingline_patdown.pop()
            yield self.hold(10) #TODO not yet defined
            self.passenger.activate()
            

env = sim.Environment(trace=True)

PassengerGenerator()
passportcontrol = PassportControl()
securityscan = SecurityScan()
patdown = PatDown()

waitingline_passport = sim.Queue('waitingline_passport')
waitingline_security = sim.Queue('waitingline_security')
waitingline_patdown = sim.Queue('waitingline_patdown')

waitingline_luggage = sim.Queue('waitingline_luggage')

env.run(till=60*60)
print()
waitingline_passport.print_statistics()
waitingline_security.print_statistics()
waitingline_patdown.print_statistics()
waitingline_luggage.print_statistics()

     0.000 main                 current                              
                                passengergenerator create            
                                passengergenerator activate          scheduled for      0.000 @process
                                passportcontrol create               
                                passportcontrol activate             scheduled for      0.000 @process
                                securityscan create                  
                                securityscan activate                scheduled for      0.000 @process
                                patdown create                       
                                patdown activate                     scheduled for      0.000 @process
                                waitingline_passport create          
                                waitingline_security create          
                                waitingline_patdown create           
                            

In [4]:
### Simulation with matching luggage to passenger

class PassengerGenerator(sim.Component):
    def process(self):
        while True:
            Passenger()
            yield self.hold(sim.Normal(60).sample())
            
class Passenger(sim.Component):
    def process(self):
        # Passport control
        self.enter(waitingline_passport)
        if passportcontrol.ispassive():
            passportcontrol.activate()
        yield self.passivate()
        
        # Walking from passport control to security scan
        yield self.hold(5/(2.5*1000/(60*60))) # convert 2.5 km/h to m/s
        
        # Security Scan
        self.enter(waitingline_security)
        if securityscan.ispassive():
            securityscan.activate()
        yield self.passivate()
        
        # Put luggage on belt
        yield self.hold(sim.Uniform(20,40).sample())
        Luggage(self.name())
        
        # 10% Pat Down Requirement
        if (sim.Uniform(1,100).sample() <= 10):
            # Walking from security scan to patdown
            yield self.hold(5/(2.5*1000/(60*60))) # convert 2.5 km/h to m/s
            # Patdown
            self.enter(waitingline_patdown)
            if patdown.ispassive():
                patdown.activate()
            yield self.passivate()
        
        # Walking along luggage belt
        yield self.hold(10/(0.5*1000/(60*60))) # convert 2.5 km/h to m/s
        
        # Pick up luggage, not matching owner
        self.enter(waitingline_passengerLuggagePickup)
        if luggagePickup.ispassive():
            luggagePickup.activate()
        yield self.passivate()
                
class Luggage(sim.Component): 
    def __init__(self, passengerName, *args, **kwargs): 
        super().__init__(*args, **kwargs)
        self.owner = passengerName
        
    def process(self):
        # Roll along luggage belt
        yield self.hold(10/(2.5*1000/(60*60))) # convert 2.5 km/h to m/s
        
        # Enter luggage waiting line
        self.enter(waitingline_luggageLuggagePickup)
        if luggagePickup.ispassive():
                luggagePickup.activate()
        self.passivate()
        
class LuggagePickup(sim.Component):
    def process(self):
        while True:
            while len(waitingline_passengerLuggagePickup) == 0 or len(waitingline_luggageLuggagePickup) == 0:
                yield self.passivate()
            for passenger in waitingline_passengerLuggagePickup:
                  for luggage in waitingline_luggageLuggagePickup: 
                        if luggage.owner == passenger.name(): 
                            #found luggage of owner!
                            waitingline_passengerLuggagePickup.remove(passenger)
                            waitingline_luggageLuggagePickup.remove(luggage)
                            
                            yield self.hold(sim.Uniform(20,40).sample())

                            passenger.activate()
                            #luggage.activate()

                            break
        
class PassportControl(sim.Component):
    def process(self):
        while True:
            while len(waitingline_passport) == 0:
                yield self.passivate()
            self.passenger = waitingline_passport.pop()
            yield self.hold(sim.Triangular(30.,90.,45.).sample())
            self.passenger.activate()
            
class SecurityScan(sim.Component):
    def process(self):
        while True:
            while len(waitingline_security) == 0:
                yield self.passivate()
            self.passenger = waitingline_security.pop()
            yield self.hold(10)
            self.passenger.activate()
            
class PatDown(sim.Component):
    def process(self):
        while True:
            while len(waitingline_patdown) == 0:
                yield self.passivate()
            self.passenger = waitingline_patdown.pop()
            yield self.hold(10) #TODO not yet defined
            self.passenger.activate()
            

env = sim.Environment(trace=True)

PassengerGenerator()
passportcontrol = PassportControl()
securityscan = SecurityScan()
patdown = PatDown()
luggagePickup = LuggagePickup()

waitingline_passport = sim.Queue('waitingline_passport')
waitingline_security = sim.Queue('waitingline_security')
waitingline_patdown = sim.Queue('waitingline_patdown')

waitingline_luggageLuggagePickup = sim.Queue('waitingline_luggageLuggagePickup')
waitingline_passengerLuggagePickup = sim.Queue('waitingline_passengerLuggagePickup')

env.run(till=60*60)
print()
waitingline_passport.print_statistics()
waitingline_security.print_statistics()
waitingline_patdown.print_statistics()
waitingline_luggage.print_statistics()

     0.000 main                 current                              
                                passengergenerator create            
                                passengergenerator activate          scheduled for      0.000 @process
                                passportcontrol create               
                                passportcontrol activate             scheduled for      0.000 @process
                                securityscan create                  
                                securityscan activate                scheduled for      0.000 @process
                                patdown create                       
                                patdown activate                     scheduled for      0.000 @process
                                luggagepickup create                 
                                luggagepickup activate               scheduled for      0.000 @process
                                waitingline_passport create      

## Working 18/09/2017

In [7]:
import salabim as sim
import time
import sys

class PassengerGenerator(sim.Component):
    def process(self):
        while True:
            Passenger()
            yield self.hold(sim.Normal(60).sample())

class Passenger(sim.Component):
    def process(self):
        # Passport control
        self.enter(waitingline_passport)
        if passportcontrol.ispassive():
            passportcontrol.activate()
        yield self.passivate()

        # Walking from passport control to security scan
        yield self.hold(5/(2.5*1000/(60*60))) # convert 2.5 km/h to m/s

        # Security Scan
        self.enter(waitingline_security)
        if securityscan.ispassive():
            securityscan.activate()
        yield self.passivate()

        # Put luggage on belt
        yield self.hold(sim.Uniform(20,40).sample())
        Luggage(self.name())

        # 10% Pat Down Requirement
        if (sim.Uniform(1,100).sample() <= 10):
            # Walking from security scan to patdown
            yield self.hold(5/(2.5*1000/(60*60))) # convert 2.5 km/h to m/s
            # Patdown
            self.enter(waitingline_patdown)
            if patdown.ispassive():
                patdown.activate()
            yield self.passivate()

        # Walking along luggage belt
        yield self.hold(10/(0.5*1000/(60*60))) # convert 2.5 km/h to m/s

        # Pick up luggage, not matching owner
        self.enter(waitingline_passengerLuggagePickup)
        if luggagePickup.ispassive():
            luggagePickup.activate()
        yield self.passivate()


class Luggage(sim.Component):
    def __init__(self, passengerName, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.owner = passengerName

    def process(self):
        # Roll along luggage belt
        yield self.hold(10/(2.5*1000/(60*60))) # convert 2.5 km/h to m/s

        # Enter luggage waiting line
        self.enter(waitingline_luggageLuggagePickup)
        #if luggagePickup.ispassive():
        #        luggagePickup.activate()
        self.passivate()

class PassportControl(sim.Component):
    def process(self):
        while True:
            while len(waitingline_passport) == 0:
                yield self.passivate()
            self.passenger = waitingline_passport.pop()
            yield self.hold(sim.Triangular(30.,90.,45.).sample())
            self.passenger.activate()

class SecurityScan(sim.Component):
    def process(self):
        while True:
            while len(waitingline_security) == 0:
                yield self.passivate()
            self.passenger = waitingline_security.pop()
            yield self.hold(10)
            self.passenger.activate()

class PatDown(sim.Component):
    def process(self):
        while True:
            while len(waitingline_patdown) == 0:
                yield self.passivate()
            self.passenger = waitingline_patdown.pop()
            yield self.hold(10) #TODO not yet defined
            self.passenger.activate()

class LuggagePickup(sim.Component):
    def process(self):
        while True:
            while len(waitingline_passengerLuggagePickup) == 0 or len(waitingline_luggageLuggagePickup) == 0:
                yield self.passivate()

            for passenger in waitingline_passengerLuggagePickup:
                for luggage in waitingline_luggageLuggagePickup:
                    if luggage.owner == passenger.name():
                        #found luggage of owner!
                        waitingline_passengerLuggagePickup.remove(passenger)
                        waitingline_luggageLuggagePickup.remove(luggage)

                        yield self.hold(sim.Uniform(20,40).sample())

                        passenger.activate()
                        #luggage.activate() This is problematic because it is never the luggage who activates successfully

                        break

#pax statistics
pax_thru_mean = 0
pax_thru_95 = 0

#queue statistics
passport_length = 0
passport_waiting = 0
luggage_drop_length = 0
luggage_drop_waiting = 0
luggage_pickup_length = 0
luggage_pickup_waiting = 0

#utilization statistics
passport_util = 0
scanner_util = 0
patdown_util = 0

replications = 10
#if len(sys.argv) > 1:
    #replications = int(sys.argv[1])

for exp in range(0,replications):
    #steipatr non-random seed for reproduceability
    env = sim.Environment(trace=False,random_seed=exp)

    PassengerGenerator()
    passportcontrol = PassportControl()
    securityscan = SecurityScan()
    patdown = PatDown()
    luggagePickup = LuggagePickup()

    waitingline_passport = sim.Queue('waitingline_passport')
    waitingline_security = sim.Queue('waitingline_security')
    waitingline_patdown = sim.Queue('waitingline_patdown')
    waitingline_luggage = sim.Queue('waitingline_luggage')

    waitingline_luggageLuggagePickup = sim.Queue('waitingline_luggageLuggagePickup')
    waitingline_passengerLuggagePickup = sim.Queue('waitingline_passengerLuggagePickup')

    # Warm-up - don't collect statistics
    waitingline_passport.length.monitor(False)
    waitingline_security.length.monitor(False)
    waitingline_patdown.length.monitor(False)
    waitingline_luggage.length.monitor(False)
    env.run(duration=60*60)

    # Collect statistics
    waitingline_passport.length.monitor(True)
    waitingline_security.length.monitor(True)
    waitingline_patdown.length.monitor(True)
    waitingline_luggage.length.monitor(True)
    env.run(duration=4*60*60)

    passport_length += waitingline_passport.length.mean()
    passport_waiting += waitingline_passport.length_of_stay.mean()
    luggage_pickup_length += waitingline_luggage.length.mean()
    luggage_pickup_waiting += waitingline_luggage.length_of_stay.mean()

print()
print("-- Pax Statistics --")
print("passenger throughput time mean:")
print("passenger throughput time 95% confidence interval:")
print()
print("-- Queue Statistics --")
print("passport queue length mean:",passport_length/replications)
print("passport queue waiting time mean [s]:",passport_waiting/replications)
print("luggage drop length mean:")
print("luggage drop waiting time mean [s]:")
print("luggage pickup length mean:",luggage_pickup_length/replications)
print("luggage pickup waiting time mean [s]:",luggage_pickup_waiting/replications)
print()
print("-- Utilization Statistics --")
print("passport control utilization:")
print("scanner utilization:")
print("patdown utilization:")
print()



-- Pax Statistics --
passenger throughput time mean:
passenger throughput time 95% confidence interval:

-- Queue Statistics --
passport queue length mean: 0.186405712442
passport queue waiting time mean [s]: 11.1099128857
luggage drop length mean:
luggage drop waiting time mean [s]:
luggage pickup length mean: 0.0
luggage pickup waiting time mean [s]: nan

-- Utilization Statistics --
passport control utilization:
scanner utilization:
patdown utilization:



In [10]:
import salabim as sim
import time
import sys

class PassengerGenerator(sim.Component):
    def process(self):
        while True:
            Passenger()
            yield self.hold(sim.Exponential(60).sample())

class Passenger(sim.Component):
    def process(self):
        self.arrivaltime = env.now()
        # Passport control
        self.enter(waitingline_passport)
        if passportControl.ispassive():
            passportControl.activate()
        yield self.passivate()

        # Walking from passport control to security scan
        yield self.hold(5/(2.5*1000/(60*60))) # convert 2.5 km/h to m/s

        # Security Scan
        self.enter(waitingline_security)
        if securityscan.ispassive():
            securityscan.activate()
        yield self.passivate()

        # Put luggage on belt
        self.enter(waitingline_luggageDropoff)
        if luggageDropoff.ispassive():
            luggageDropoff.activate()
        yield self.passivate()

        # 10% Pat Down Requirement
        if (sim.Uniform(1,100).sample() <= 10):
            # Walking from security scan to patdown
            yield self.hold(5/(2.5*1000/(60*60))) # convert 2.5 km/h to m/s
            # Patdown
            self.enter(waitingline_patdown)
            if patdown.ispassive():
                patdown.activate()
            yield self.passivate()

        # Walking along luggage belt
        yield self.hold(10/(0.5*1000/(60*60))) # convert 2.5 km/h to m/s

        # Pick up luggage, matching owner
        self.enter(waitingline_passengerLuggagePickup)
        if luggagePickup.ispassive():
            luggagePickup.activate()
        yield self.passivate()
        luggagePickup.monitor_time_in_complex.tally(env.now() - self.arrivaltime)


class Luggage(sim.Component):
    def __init__(self, passengerName, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.owner = passengerName

    def process(self):
        # Roll along luggage belt
        yield self.hold(10/(2.5*1000/(60*60))) # convert 2.5 km/h to m/s

        # Enter luggage waiting line
        self.enter(waitingline_luggageLuggagePickup)
        #if luggagePickup.ispassive():
        #        luggagePickup.activate()
        self.passivate()

class PassportControl(sim.Component):
    def process(self):
        while True:
            while len(waitingline_passport) == 0:
                yield self.passivate()
            self.passenger = waitingline_passport.pop()
            yield self.hold(sim.Uniform(20,40).sample())
            self.passenger.activate()

class LuggageDropoff(sim.Component):
    def process(self):
        while True:
            while len(waitingline_luggageDropoff) == 0:
                yield self.passivate()

            self.passenger = waitingline_luggageDropoff.pop()

            Luggage(self.passenger.name())
            yield self.hold(sim.Triangular(30.,90.,45.).sample())
            self.passenger.activate()

class SecurityScan(sim.Component):
    def process(self):
        while True:
            while len(waitingline_security) == 0:
                yield self.passivate()
            self.passenger = waitingline_security.pop()
            yield self.hold(10)
            self.passenger.activate()

class PatDown(sim.Component):
    def process(self):
        while True:
            while len(waitingline_patdown) == 0:
                yield self.passivate()
            self.passenger = waitingline_patdown.pop()
            yield self.hold(sim.Uniform(60,120).sample())
            self.passenger.activate()

class LuggagePickup(sim.Component):
    def __init__(self, *args, **kwargs):
      super().__init__(*args, **kwargs)
      self.monitor_time_in_complex = sim.Monitor(name='time in complex')
  
    def process(self):
        while True:
            while len(waitingline_passengerLuggagePickup) == 0 or len(waitingline_luggageLuggagePickup) == 0:
                yield self.passivate()

            for passenger in waitingline_passengerLuggagePickup:
                #print(passenger.name())
                for luggage in waitingline_luggageLuggagePickup:
                    #print(luggage.name())
                    if luggage.owner == passenger.name():
                        #found luggage of owner!
                        waitingline_passengerLuggagePickup.remove(passenger)
                        waitingline_luggageLuggagePickup.remove(luggage)

                        yield self.hold(sim.Uniform(20,40).sample())

                        passenger.activate()
                        #luggage.activate() This is problematic because it is never the luggage who activates successfully

                        break

            yield self.passivate()

#pax statistics
pax_thru_mean = 0
pax_thru_95 = 0

#queue statistics
passport_length = 0
passport_waiting = 0
luggage_drop_length = 0
luggage_drop_waiting = 0
luggage_pickup_length = 0
luggage_pickup_waiting = 0

#utilization statistics
passport_util = 0
scanner_util = 0
patdown_util = 0

replications = 10
#if len(sys.argv) > 1:
    #replications = int(sys.argv[1])

for exp in range(0,replications):
    #steipatr non-random seed for reproduceability
    env = sim.Environment(trace=False,random_seed=exp)

    PassengerGenerator()
    passportControl = PassportControl()
    luggageDropoff = LuggageDropoff()
    securityscan = SecurityScan()
    patdown = PatDown()
    luggagePickup = LuggagePickup()

    waitingline_passport = sim.Queue('waitingline_passport')
    waitingline_security = sim.Queue('waitingline_security')
    waitingline_patdown = sim.Queue('waitingline_patdown')
    waitingline_luggageDropoff = sim.Queue('waitingline_luggageDropoff')
    waitingline_luggageLuggagePickup = sim.Queue('waitingline_luggageLuggagePickup')
    waitingline_passengerLuggagePickup = sim.Queue('waitingline_passengerLuggagePickup')

    #TODO why suspend only length monitoring? Does everything need to be suspended?
    # Warm-up - don't collect statistics
    waitingline_passport.length.monitor(False)
    waitingline_security.length.monitor(False)
    waitingline_patdown.length.monitor(False)
    waitingline_luggageDropoff.length.monitor(False)
    waitingline_luggageLuggagePickup.length.monitor(False)
    waitingline_passengerLuggagePickup.length.monitor(False)
    env.run(duration=60*60)

    # Collect statistics
    waitingline_passport.length.monitor(True)
    waitingline_security.length.monitor(True)
    waitingline_patdown.length.monitor(True)
    waitingline_luggageDropoff.length.monitor(True)
    waitingline_luggageLuggagePickup.length.monitor(True)
    waitingline_passengerLuggagePickup.length.monitor(True)
    env.run(duration=4*60*60)

    pax_thru_mean += luggagePickup.monitor_time_in_complex.mean()
    pax_thru_95 += luggagePickup.monitor_time_in_complex.percentile(95)
    
    passport_length += waitingline_passport.length.mean()
    passport_waiting += waitingline_passport.length_of_stay.mean()
    luggage_drop_length += waitingline_luggageDropoff.length.mean()
    luggage_drop_waiting += waitingline_luggageDropoff.length_of_stay.mean()
    luggage_pickup_length += waitingline_passengerLuggagePickup.length.mean()
    luggage_pickup_waiting += waitingline_passengerLuggagePickup.length_of_stay.mean()
    

print()
print("-- Pax Statistics --")
print("passenger throughput time mean [s]:",pax_thru_mean/replications, "seconds")
#TODO why not use pax_thru_95 here?
print("passenger throughput time 95% confidence interval [s]:",luggagePickup.monitor_time_in_complex.percentile(95)/replications)
print()
print("-- Queue Statistics --")
print("passport queue length mean:",passport_length/replications)
print("passport queue waiting time mean [s]:",passport_waiting/replications)
print("luggage drop length mean:",luggage_drop_length/replications)
print("luggage drop waiting time mean [s]:",luggage_drop_waiting/replications)
print("luggage pickup length mean:",luggage_pickup_length/replications)
print("luggage pickup waiting time mean [s]:",luggage_pickup_waiting/replications)
print()
print("-- Utilization Statistics --")
print("passport control utilization:")
print("scanner utilization:")
print("patdown utilization:")
print()



-- Pax Statistics --
passenger throughput time mean [s]: 417.366284822 seconds
passenger throughput time 95% confidence interval [s]: 53.7432329358

-- Queue Statistics --
passport queue length mean: 0.262186265975
passport queue waiting time mean [s]: 14.9198601148
luggage drop length mean: 3.44650621456
luggage drop waiting time mean [s]: 176.145579795
luggage pickup length mean: 0.644492732735
luggage pickup waiting time mean [s]: 14.1317183642

-- Utilization Statistics --
passport control utilization:
scanner utilization:
patdown utilization:



In [12]:
60+45+7.2*2++10+45+0.9*14.4+0.1*90+45

241.36

In [70]:
#edited by steipatr at 15:23
import salabim as sim
import time, sys

class PassengerGenerator(sim.Component):
    def process(self):
        while True:
            Passenger()
            yield self.hold(sim.Exponential(60).sample())

class Passenger(sim.Component):
    def process(self):
        self.arrivaltime = env.now()
        # Passport control
        self.enter(waitingline_passport)
        if passportControl.ispassive():
            passportControl.activate()
        yield self.passivate()

        # Walking from passport control to luggage belt
        yield self.hold(5/(2.5*1000/(60*60))) # convert 2.5 km/h to m/s

        # Put luggage on belt
        self.enter(waitingline_luggageDropoff)
        if luggageDropoff.ispassive():
            luggageDropoff.activate()
        yield self.passivate()
        
        # Walking from luggage belt to security scan
        yield self.hold(5/(2.5*1000/(60*60))) # convert 2.5 km/h to m/s
        
        # Security scan
        self.enter(waitingline_security)
        if securityScan.ispassive():
            securityScan.activate()
        yield self.passivate()
        
        # 10% Pat Down Requirement
        if (sim.Uniform(1,100).sample() <= 10):
            # Walking from security scan to patdown
            yield self.hold(5/(2.5*1000/(60*60))) # convert 2.5 km/h to m/s
            # Patdown
            self.enter(waitingline_patdown)
            if patDown.ispassive():
                patDown.activate()
            yield self.passivate()

        # Walking along luggage belt
        yield self.hold(10/(0.5*1000/(60*60))) # convert 2.5 km/h to m/s

        # Pick up luggage, matching owner
        self.enter(waitingline_passengerLuggagePickup)
        if luggagePickup.ispassive():
            luggagePickup.activate()
        yield self.passivate()
        luggagePickup.monitor_time_in_complex.tally(env.now() - self.arrivaltime)
        yield self.hold(sim.Uniform(20,40).sample())
        #waitingline_luggageLuggagePickup.pop()
        #waitingline_luggagePassengerPickup.pop()

class Luggage(sim.Component):
    def __init__(self, passengerName, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.owner = passengerName

    def process(self):
        # Roll along luggage belt
        yield self.hold(20) # 0.5 m/s along 10m belt

        # Enter luggage waiting line
        self.enter(waitingline_luggageLuggagePickup)
        #if luggagePickup.ispassive():
        #        luggagePickup.activate()
        self.passivate()

class Server(sim.Component):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.activeTime = 0
        self.startProcessTime = -1

    def startUtilTime(self):
        self.startProcessTime = env.now()

    def endUtilTime(self):
        if self.startProcessTime >= 0:
            endNow = env.now()
            self.activeTime += endNow - self.startProcessTime
            self.startProcessTime = -1

    def getUtilization(self):
        return self.activeTime / (env.now() - self._creation_time)

class PassportControl(Server):
    def process(self):
        while True:
            while len(waitingline_passport) == 0:
                self.endUtilTime()
                yield self.passivate()

            self.startUtilTime()
            self.passenger = waitingline_passport.pop()
            yield self.hold(sim.Triangular(30.,90.,45.).sample())
            self.passenger.activate()

class LuggageDropoff(Server):
    def process(self):
        while True:
            while len(waitingline_luggageDropoff) == 0:
                yield self.passivate()

            self.passenger = waitingline_luggageDropoff.pop()

            Luggage(self.passenger.name())
            yield self.hold(sim.Uniform(20,40).sample())
            self.passenger.activate()

class SecurityScan(Server):
    def process(self):
        while True:
            while len(waitingline_security) == 0:
                self.endUtilTime()
                yield self.passivate()

            self.startUtilTime()
            self.passenger = waitingline_security.pop()
            yield self.hold(10)
            self.passenger.activate()

class PatDown(Server):
    def process(self):
        while True:
            while len(waitingline_patdown) == 0:
                self.endUtilTime()
                yield self.passivate()

            self.startUtilTime()
            self.passenger = waitingline_patdown.pop()
            yield self.hold(sim.Uniform(60,120).sample())
            self.passenger.activate()

class LuggagePickup(Server):
    def __init__(self, *args, **kwargs):
      super().__init__(*args, **kwargs)
      self.monitor_time_in_complex = sim.Monitor(name='time in complex')

    def process(self):
        noMatches = False

        while True:
            while len(waitingline_passengerLuggagePickup) == 0 or len(waitingline_luggageLuggagePickup) == 0 or noMatches:
                yield self.passivate()

            for passenger in waitingline_passengerLuggagePickup:
                #print(passenger.name())
                for luggage in waitingline_luggageLuggagePickup:
                    #print(luggage.name())
                    if luggage.owner == passenger.name():
                        #found luggage of owner!
                        waitingline_passengerLuggagePickup.remove(passenger)
                        waitingline_luggageLuggagePickup.remove(luggage)
                        yield self.hold(sim.Uniform(20,40).sample())
                        passenger.activate()
                        #luggage.activate() This is problematic because it is never the luggage who activates successfully

                        break

                        
                        
            noMatches = True

def avg(lst):
    return sum(lst)/len(lst)
            
#pax statistics
pax_thru_mean = 0
pax_thru_95 = 0

#queue statistics
passport_length = 0
passport_waiting = 0
luggage_drop_length = 0
luggage_drop_waiting = 0
luggage_pickup_length = 0
luggage_pickup_waiting = 0

#utilization statistics
passport_util = 0
scanner_util = 0
patdown_util = 0

replications = 10
#if len(sys.argv) > 1:
#    replications = int(sys.argv[1])

for exp in range(0,replications):
    #steipatr non-random seed for reproduceability
    env = sim.Environment(trace=False,random_seed=exp)

    PassengerGenerator()
    passportControl = PassportControl()
    luggageDropoff = LuggageDropoff()
    securityScan = SecurityScan()
    patDown = PatDown()
    luggagePickup = LuggagePickup()

    waitingline_passport = sim.Queue('waitingline_passport')
    waitingline_security = sim.Queue('waitingline_security')
    waitingline_patdown = sim.Queue('waitingline_patdown')
    waitingline_luggageDropoff = sim.Queue('waitingline_luggageDropoff')
    waitingline_luggageLuggagePickup = sim.Queue('waitingline_luggageLuggagePickup')
    waitingline_passengerLuggagePickup = sim.Queue('waitingline_passengerLuggagePickup')

    #TODO why suspend only length monitoring? Does everything need to be suspended?
    # Warm-up - don't collect statistics
    waitingline_passport.length.monitor(False)
    waitingline_security.length.monitor(False)
    waitingline_patdown.length.monitor(False)
    waitingline_luggageDropoff.length.monitor(False)
    waitingline_luggageLuggagePickup.length.monitor(False)
    waitingline_passengerLuggagePickup.length.monitor(False)
    env.run(duration=60*60)

    # Collect statistics
    waitingline_passport.length.monitor(True)
    waitingline_security.length.monitor(True)
    waitingline_patdown.length.monitor(True)
    waitingline_luggageDropoff.length.monitor(True)
    waitingline_luggageLuggagePickup.length.monitor(True)
    waitingline_passengerLuggagePickup.length.monitor(True)
    env.run(duration=4*60*60)
    
    print('sum',pax_thru_mean)
    print('mean',luggagePickup.monitor_time_in_complex.mean())
    pax_thru_mean += luggagePickup.monitor_time_in_complex.mean()
    pax_thru_95 += luggagePickup.monitor_time_in_complex.percentile(95)

    passport_length += waitingline_passport.length.mean()
    passport_waiting += waitingline_passport.length_of_stay.mean()
    luggage_drop_length += waitingline_luggageDropoff.length.mean()
    luggage_drop_waiting += waitingline_luggageDropoff.length_of_stay.mean()
    luggage_pickup_length += waitingline_passengerLuggagePickup.length.mean()
    luggage_pickup_waiting += waitingline_passengerLuggagePickup.length_of_stay.mean()

print()
print("-- Pax Statistics --")
print("passenger throughput time mean [s]:", pax_thru_mean) #luggagePickup.monitor_time_in_complex.mean()) #pax_thru_mean/replications
#TODO why not use pax_thru_95 here?
print("passenger throughput time 95% confidence interval [s]:",luggagePickup.monitor_time_in_complex.percentile(95)/replications)
print()
print("-- Queue Statistics --")
print("passport queue length mean:",passport_length/replications)
print("passport queue waiting time mean [s]:",passport_waiting/replications)
print("luggage drop length mean:",luggage_drop_length/replications)
print("luggage drop waiting time mean [s]:",luggage_drop_waiting/replications)
print("luggage pickup length mean:",luggage_pickup_length/replications)
print("luggage pickup waiting time mean [s]:",luggage_pickup_waiting/replications)
print()
print("-- Utilization Statistics --")
print("passport control utilization:",passportControl.getUtilization())
print("scanner utilization:",securityScan.getUtilization())
print("patdown utilization:",patDown.getUtilization())
print()

sum 0
mean nan
sum nan
mean 224.920413738
sum nan
mean nan
sum nan
mean 202.158002083
sum nan
mean 186.191327088
sum nan
mean 217.058016689
sum nan
mean nan
sum nan
mean 210.729422541
sum nan
mean 237.869477917
sum nan
mean 204.36143001

-- Pax Statistics --
passenger throughput time mean [s]: nan
passenger throughput time 95% confidence interval [s]: 20.436143001

-- Queue Statistics --
passport queue length mean: 3.78590955797
passport queue waiting time mean [s]: 207.834939289
luggage drop length mean: 0.000876562861621
luggage drop waiting time mean [s]: 0.052364568861
luggage pickup length mean: 167.140965895
luggage pickup waiting time mean [s]: nan

-- Utilization Statistics --
passport control utilization: 0.08102153420255453
scanner utilization: 0.1638888888888889
patdown utilization: 0.16344041578190602



In [76]:
print(pax_thru_mean)
print(replications)
print(pax_thru_95)
a=[3,45,5,123,4,5]
avg(a)

nan
10
nan


30.833333333333332

In [62]:
luggagePickup.monitor_time_in_complex.mean()

204.36143001008739

In [58]:
monitor_time_in_complex

NameError: name 'monitor_time_in_complex' is not defined

In [32]:
LuggagePickup

__main__.LuggagePickup

In [53]:
luggagePickup.monitor_time_in_complex

In [88]:
#edited by steipatr at 15:23
import salabim as sim
import time, sys

class PassengerGenerator(sim.Component):
    def process(self):
        while True:
            Passenger()
            yield self.hold(sim.Exponential(60).sample())

class Passenger(sim.Component):
    def process(self):
        self.arrivaltime = env.now()
        # Passport control
        self.enter(waitingline_passport)
        if passportControl.ispassive():
            passportControl.activate()
        yield self.passivate()

        # Walking from passport control to luggage belt
        yield self.hold(5/(2.5*1000/(60*60))) # convert 2.5 km/h to m/s

        # Put luggage on belt
        self.enter(waitingline_luggageDropoff)
        if luggageDropoff.ispassive():
            luggageDropoff.activate()
        yield self.passivate()
        
        # Walking from luggage belt to security scan
        yield self.hold(5/(2.5*1000/(60*60))) # convert 2.5 km/h to m/s
        
        # Security scan
        self.enter(waitingline_security)
        if securityScan.ispassive():
            securityScan.activate()
        yield self.passivate()
        
        # 10% Pat Down Requirement
        if (sim.Uniform(1,100).sample() <= 10):
            # Walking from security scan to patdown
            yield self.hold(5/(2.5*1000/(60*60))) # convert 2.5 km/h to m/s
            # Patdown
            self.enter(waitingline_patdown)
            if patDown.ispassive():
                patDown.activate()
            yield self.passivate()

        # Walking along luggage belt
        yield self.hold(10/(0.5*1000/(60*60))) # convert 2.5 km/h to m/s

        # Pick up luggage, matching owner
        self.enter(waitingline_passengerLuggagePickup)
        if luggagePickup.ispassive():
            luggagePickup.activate()
        yield self.passivate()
        luggagePickup.monitor_time_in_complex.tally(env.now() - self.arrivaltime)
        yield self.hold(sim.Uniform(20,40).sample())
        #waitingline_luggageLuggagePickup.pop()
        #waitingline_luggagePassengerPickup.pop()

class Luggage(sim.Component):
    def __init__(self, passengerName, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.owner = passengerName

    def process(self):
        # Roll along luggage belt
        yield self.hold(20) # 0.5 m/s along 10m belt

        # Enter luggage waiting line
        self.enter(waitingline_luggageLuggagePickup)
        #if luggagePickup.ispassive():
        #        luggagePickup.activate()
        self.passivate()

class Server(sim.Component):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.activeTime = 0
        self.startProcessTime = -1

    def startUtilTime(self):
        self.startProcessTime = env.now()

    def endUtilTime(self):
        if self.startProcessTime >= 0:
            endNow = env.now()
            self.activeTime += endNow - self.startProcessTime
            self.startProcessTime = -1

    def getUtilization(self):
        return self.activeTime / (env.now() - self._creation_time)

class PassportControl(Server):
    def process(self):
        while True:
            while len(waitingline_passport) == 0:
                self.endUtilTime()
                yield self.passivate()

            self.startUtilTime()
            self.passenger = waitingline_passport.pop()
            yield self.hold(sim.Triangular(30.,90.,45.).sample())
            self.passenger.activate()

class LuggageDropoff(Server):
    def process(self):
        while True:
            while len(waitingline_luggageDropoff) == 0:
                yield self.passivate()

            self.passenger = waitingline_luggageDropoff.pop()

            Luggage(self.passenger.name())
            yield self.hold(sim.Uniform(20,40).sample())
            self.passenger.activate()

class SecurityScan(Server):
    def process(self):
        while True:
            while len(waitingline_security) == 0:
                self.endUtilTime()
                yield self.passivate()

            self.startUtilTime()
            self.passenger = waitingline_security.pop()
            yield self.hold(10)
            self.passenger.activate()

class PatDown(Server):
    def process(self):
        while True:
            while len(waitingline_patdown) == 0:
                self.endUtilTime()
                yield self.passivate()

            self.startUtilTime()
            self.passenger = waitingline_patdown.pop()
            yield self.hold(sim.Uniform(60,120).sample())
            self.passenger.activate()

class LuggagePickup(Server):
    def __init__(self, *args, **kwargs):
      super().__init__(*args, **kwargs)
      self.monitor_time_in_complex = sim.Monitor(name='time in complex')

    def process(self):
        noMatches = False

        while True:
            while len(waitingline_passengerLuggagePickup) == 0 or len(waitingline_luggageLuggagePickup) == 0 or noMatches:
                yield self.passivate()

            for passenger in waitingline_passengerLuggagePickup:
                #print(passenger.name())
                for luggage in waitingline_luggageLuggagePickup:
                    #print(luggage.name())
                    if luggage.owner == passenger.name():
                        #found luggage of owner!
                        waitingline_passengerLuggagePickup.remove(passenger)
                        waitingline_luggageLuggagePickup.remove(luggage)
                        yield self.hold(sim.Uniform(20,40).sample())
                        passenger.activate()
                        #luggage.activate() This is problematic because it is never the luggage who activates successfully

                        break

                        
                        
            noMatches = True

def avg(lst):
    cleanedlst = [x for x in lst if str(x) != 'nan']
    if len(cleanedlst)==0:
        return 0
    return sum(cleanedlst)/len(cleanedlst)
            
#pax statistics
pax_thru_mean = []
pax_thru_95 = []

#queue statistics
passport_length = 0
passport_waiting = 0
luggage_drop_length = 0
luggage_drop_waiting = 0
luggage_pickup_length = 0
luggage_pickup_waiting = 0

#utilization statistics
passport_util = 0
scanner_util = 0
patdown_util = 0

replications = 10
#if len(sys.argv) > 1:
#    replications = int(sys.argv[1])

for exp in range(0,replications):
    #steipatr non-random seed for reproduceability
    env = sim.Environment(trace=False,random_seed=exp)

    PassengerGenerator()
    passportControl = PassportControl()
    luggageDropoff = LuggageDropoff()
    securityScan = SecurityScan()
    patDown = PatDown()
    luggagePickup = LuggagePickup()

    waitingline_passport = sim.Queue('waitingline_passport')
    waitingline_security = sim.Queue('waitingline_security')
    waitingline_patdown = sim.Queue('waitingline_patdown')
    waitingline_luggageDropoff = sim.Queue('waitingline_luggageDropoff')
    waitingline_luggageLuggagePickup = sim.Queue('waitingline_luggageLuggagePickup')
    waitingline_passengerLuggagePickup = sim.Queue('waitingline_passengerLuggagePickup')

    #TODO why suspend only length monitoring? Does everything need to be suspended?
    # Warm-up - don't collect statistics
    waitingline_passport.length.monitor(False)
    waitingline_security.length.monitor(False)
    waitingline_patdown.length.monitor(False)
    waitingline_luggageDropoff.length.monitor(False)
    waitingline_luggageLuggagePickup.length.monitor(False)
    waitingline_passengerLuggagePickup.length.monitor(False)
    env.run(duration=60*60)

    # Collect statistics
    waitingline_passport.length.monitor(True)
    waitingline_security.length.monitor(True)
    waitingline_patdown.length.monitor(True)
    waitingline_luggageDropoff.length.monitor(True)
    waitingline_luggageLuggagePickup.length.monitor(True)
    waitingline_passengerLuggagePickup.length.monitor(True)
    env.run(duration=4*60*60)
    
    #print('list',pax_thru_mean)
    #print('mean',luggagePickup.monitor_time_in_complex.mean())
    #print('avglst',avg(pax_thru_mean))
    #print('95percentile',pax_thru_95)
    pax_thru_mean += [luggagePickup.monitor_time_in_complex.mean()]
    pax_thru_95 += [luggagePickup.monitor_time_in_complex.percentile(95)]

    passport_length += waitingline_passport.length.mean()
    passport_waiting += waitingline_passport.length_of_stay.mean()
    luggage_drop_length += waitingline_luggageDropoff.length.mean()
    luggage_drop_waiting += waitingline_luggageDropoff.length_of_stay.mean()
    luggage_pickup_length += waitingline_passengerLuggagePickup.length.mean()
    luggage_pickup_waiting += waitingline_passengerLuggagePickup.length_of_stay.mean()

print()
print("-- Pax Statistics --")
print("passenger throughput time mean [s]:", avg(pax_thru_mean)) #luggagePickup.monitor_time_in_complex.mean()) #pax_thru_mean/replications
#TODO why not use pax_thru_95 here?
print("passenger throughput time 95% confidence interval [s]:", avg(pax_thru_95))#luggagePickup.monitor_time_in_complex.percentile(95)/replications)
print()
print("-- Queue Statistics --")
print("passport queue length mean:",passport_length/replications)
print("passport queue waiting time mean [s]:",passport_waiting/replications)
print("luggage drop length mean:",luggage_drop_length/replications)
print("luggage drop waiting time mean [s]:",luggage_drop_waiting/replications)
print("luggage pickup length mean:",luggage_pickup_length/replications)
print("luggage pickup waiting time mean [s]:",luggage_pickup_waiting/replications)
print()
print("-- Utilization Statistics --")
print("passport control utilization:",passportControl.getUtilization())
print("scanner utilization:",securityScan.getUtilization())
print("patdown utilization:",patDown.getUtilization())
print()

avglst 0
95percentile []
avglst 0
95percentile [nan]
avglst 224.920413738
95percentile [nan, 224.92041373811875]
avglst 224.920413738
95percentile [nan, 224.92041373811875, nan]
avglst 213.539207911
95percentile [nan, 224.92041373811875, nan, 202.15800208325837]
avglst 204.423247636
95percentile [nan, 224.92041373811875, nan, 202.15800208325837, 186.19132708801803]
avglst 207.5819399
95percentile [nan, 224.92041373811875, nan, 202.15800208325837, 186.19132708801803, 217.05801668941922]
avglst 207.5819399
95percentile [nan, 224.92041373811875, nan, 202.15800208325837, 186.19132708801803, 217.05801668941922, nan]
avglst 208.211436428
95percentile [nan, 224.92041373811875, nan, 202.15800208325837, 186.19132708801803, 217.05801668941922, nan, 210.72942254064509]
avglst 213.154443343
95percentile [nan, 224.92041373811875, nan, 202.15800208325837, 186.19132708801803, 217.05801668941922, nan, 210.72942254064509, 237.86947791688056]

-- Pax Statistics --
passenger throughput time mean [s]: 211